---
## Setup

### Mount Bucket

In [ ]:
from pyspark.sql.functions import *
import urllib


file_type = "csv"
first_row_is_header = "true"
delimiter = ","
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

AWS_S3_BUCKET = "user-0ea903d23769-bucket"
MOUNT_NAME = "/mnt/0ea903d23769-bucket"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

### Read Topic Contents Into Dataframes

In [ ]:
def read_data(spark, file_type, infer_schema, location):
    return spark.read.format(file_type) \
                    .option("inferSchema", infer_schema) \
                    .load(location)

file_type = "json"
infer_schema = "true"

location_pin = "/mnt/0ea903d23769-bucket/topics/0ea903d23769.pin/partition=0/*.json"
location_geo = "/mnt/0ea903d23769-bucket/topics/0ea903d23769.geo/partition=0/*.json"
location_user = "/mnt/0ea903d23769-bucket/topics/0ea903d23769.user/partition=0/*.json"

df_pin_dirty = read_data(spark, file_type, infer_schema, location_pin)
df_geo_dirty = read_data(spark, file_type, infer_schema, location_geo)
df_user_dirty = read_data(spark, file_type, infer_schema, location_user)


### Download Dirty CSVs

In [ ]:
# df_pin.write.csv('FileStore/tables/pin_dirty.csv')
# df_geo.write.csv('FileStore/tables/geo_dirty.csv')
# df_user.write.csv('FileStore/tables/user_dirty.csv')

display(df_pin_dirty.select("*"))
display(df_geo_dirty.select("*"))
display(df_use_dirty.select("*"))

### Create Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, TimestampType, DateType

spark = SparkSession.builder.appName("DF Cleaning").getOrCreate()

---
## Cleaning

### Cleaning df_pin

In [ ]:
df_pin = df_pin_dirty

# Replace empty entries and entries with no relevant data in each column with Nones
df_pin = df_pin.replace(['', ' ', 'NULL', 'null'], [None] * 4)
df_pin = df_pin.withColumn("description", when(col("description") == "No description available Story format", None).otherwise(col("description")))
df_pin = df_pin.withColumn("follower_count", when(col("follower_count") == "User Info Error", None).otherwise(col("follower_count")))
df_pin = df_pin.withColumn("image_src", when(col("image_src") == "Image src error.", None).otherwise(col("image_src")))
df_pin = df_pin.withColumn("poster_name", when(col("poster_name") == "User Info Error", None).otherwise(col("poster_name")))
df_pin = df_pin.withColumn("tag_list", when(col("tag_list") == "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e", None).otherwise(col("tag_list")))
df_pin = df_pin.withColumn("title", when(col("title") == "No Title Data Available", None).otherwise(col("title")))

# Transform follower_count to ensure every entry is a number and data type is an int
# Remove any non-numeric characters (like 'k' in '136k') and then convert to integer
df_pin = df_pin.withColumn("follower_count", regexp_replace(col("follower_count"), "[^0-9]", ""))
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast(IntegerType()))

# Ensure that each column containing numeric data has a numeric data type
df_pin = df_pin.withColumn("downloaded", col("downloaded").cast(IntegerType()))

# Clean the data in the save_location column to include only the save location path
df_pin = df_pin.withColumn("save_location", regexp_replace(col("save_location"), "Local save in ", ""))

# Rename the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")

# Reorder the DataFrame columns
df_pin = df_pin.select(["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"])


display(df_pin.select("*"))

### Cleaning df_geo

In [ ]:
df_geo = df_geo_dirty

# Create a new column 'coordinates' containing an array of latitude and longitude
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Drop the latitude and longitude columns
df_geo = df_geo.drop("latitude", "longitude")

# Convert the timestamp column from a string to a timestamp data type
df_geo = df_geo.withColumn("timestamp", col("timestamp").cast(TimestampType()))

# Reorder the DataFrame columns
df_geo = df_geo.select(["ind", "country", "coordinates", "timestamp"])

display(df_geo.select("*"))

### Cleaning df_user

In [ ]:
df_user = df_user_dirty

# Create a new column 'user_name' by concatenating 'first_name' and 'last_name'
df_user = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))

# Drop the 'first_name' and 'last_name' columns
df_user = df_user.drop("first_name", "last_name")

# Convert the 'date_joined' column from a string to a timestamp data type
df_user = df_user.withColumn("date_joined", col("date_joined").cast(TimestampType()))

# Reorder the DataFrame columns
df_user = df_user.select(["ind", "user_name", "age", "date_joined"])

display(df_user.select("*"))

---
## Tests

### Find Most Popular Category per Country

In [ ]:
# Join the geo and pin DataFrames on the 'ind' column
df_joined = df_geo.join(df_pin, "ind")

# Group by country and category and count the occurrences
df_category_count = df_joined.groupBy("country", "category").agg(count("*").alias("category_count"))

# Find the most popular category for each country by sorting within each group
window = Window.partitionBy("country").orderBy(col("category_count").desc())

df_most_popular = df_category_count.withColumn("rank", rank().over(window)) \
                                   .filter(col("rank") == 1) \
                                   .drop("rank")

# Select the desired columns for the final DataFrame
df_final = df_most_popular.select("country", "category", "category_count")

display(df_final.select("*"))

### Find Post Count per Category Between 2018 & 2022

In [ ]:
df_joined = df_pin.join(df_geo, 'ind', 'inner')

# Convert the timestamp column from string to timestamp type if it's not already
df_joined = df_joined.withColumn("timestamp", col("timestamp").cast("timestamp"))

# Filter the DataFrame for posts between 2018 and 2022
df_filtered = df_joined.filter((year("timestamp") >= 2018) & (year("timestamp") <= 2022))

# Create a new column with just the year from the timestamp
df_with_year = df_filtered.withColumn("post_year", year("timestamp"))

# Group by post_year and category and count the occurrences
df_category_count = df_with_year.groupBy("post_year", "category").agg(count("*").alias("category_count"))

# Order the result for better readability
df_result = df_category_count.orderBy("post_year", "category")

display(df_result.select("*"))

### Find Most Followed User per Country

In [ ]:
df_joined = df_pin.join(df_geo, 'ind', 'inner')

# Define a window spec partitioned by country
windowSpec = Window.partitionBy("country").orderBy(col("follower_count").desc())

# Use the window spec to add a row number for each user within each country partition
df_ranked = df_joined.withColumn("row_number", row_number().over(windowSpec))

# Filter for the top user (row_number 1) in each country
df_top_user_per_country = df_ranked.filter(col("row_number") == 1) \
                                   .select("country", "poster_name", "follower_count")

display(df_top_user_per_country.select("*"))

### Find The Country With Most Followed User

In [ ]:
# Find the maximum follower count across all countries
max_global_follower_count = df_top_followers_per_country.agg(max("follower_count")).collect()[0][0]

# Find the country or countries with the user that has the maximum global follower count
df_country_with_top_follower = df_top_followers_per_country.filter(col("follower_count") == max_global_follower_count) \
                                                            .select("country", "follower_count")

# Display the results
display(df_country_with_top_follower.select("*"))

### Find Most Popular Category per Age Group

In [ ]:
df_joined = df_pin.join(df_user, 'ind', 'inner')

# Create the age_group column
df_with_age_group = df_joined.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+")
)

# Group by age_group and category and count the occurrences
df_category_count = df_with_age_group.groupBy("age_group", "category").agg(count("*").alias("category_count"))

# Define a window spec partitioned by age_group and ordered by category_count descending
windowSpec = Window.partitionBy("age_group").orderBy(col("category_count").desc())

# Use the window spec to add a rank for each category within each age group partition
df_ranked = df_category_count.withColumn("rank", rank().over(windowSpec))

# Filter for the top-ranked category within each age group
df_top_category_per_age_group = df_ranked.filter(col("rank") == 1).select("age_group", "category", "category_count")

display(df_top_category_per_age_group)

### Find Median Follower Count per Age Group

In [ ]:
df_joined = df_pin.join(df_user, 'ind', 'inner')

# Age groups
df_with_age_group = df_joined.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+")
)

# Group by age_group and calculate the median follower count
df_median_follower_count = df_with_age_group.groupBy("age_group")\
                                            .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

# Display the result
display(df_median_follower_count)


### Find New User Count Between 2015 & 2020

In [ ]:
# Convert the date_joined column from string to date type
df_user = df_user.withColumn("date_joined", col("date_joined").cast(DateType()))

# Extract the year from the date_joined column
df_with_year = df_user.withColumn("post_year", year(col("date_joined")))

# Filter the DataFrame for years between 2015 and 2020
df_filtered = df_with_year.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Group by post_year and count the number of users
df_number_users_joined = df_filtered.groupBy("post_year").agg(count("*").alias("number_users_joined"))

display(df_number_users_joined)

### Find Median Follower Count of Users Joined Between 2015 & 2020

In [ ]:
# Filter users who joined between 2015 and 2020
df_filtered_users = df_user.withColumn("date_joined", col("date_joined").cast("timestamp")) \
                           .withColumn("post_year", year("date_joined")) \
                           .filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

df_joined = df_filtered_users.join(df_pin, 'ind', 'inner')

# Calculate the median follower count per post year
df_median_follower_count = df_joined.groupBy("post_year") \
                                    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

display(df_median_follower_count)

### Find Median Follower Count per Joining Year & Age Group

In [ ]:
# Filter users who joined between 2015 and 2020
df_users_filtered = df_user.withColumn("date_joined", col("date_joined").cast("timestamp")) \
                           .withColumn("post_year", year(col("date_joined"))) \
                           .filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Create the age_group column
df_users_age_grouped = df_users_filtered.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .otherwise("50+")
)

# Join df_users_age_grouped with df_pin on the user identifier to get follower counts
df_joined = df_users_age_grouped.join(df_pin, 'ind', 'inner')

# Group by age_group and post_year, and calculate the median follower count
df_median_followers = df_joined.groupBy("age_group", "post_year") \
                               .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

display(df_median_followers)